In [1]:
from pynq import Overlay
from pynq import MMIO
import numpy as np

# Program bitstream to FPGA
overlay = Overlay('/home/xilinx/workspace/part_7.bit')

# Access to memory map of the AXI GPIO 0
GPIO0_ADDR_BASE = 0x41200000
GPIO0_ADDR_RANGE = 0x10000
gpio0_obj = MMIO(GPIO0_ADDR_BASE, GPIO0_ADDR_RANGE)

# Access to memory map of the BRAM 0
BRAM0_ADDR_BASE = 0x40000000
BRAM0_ADDR_RANGE = 0x2000
bram0_obj = MMIO(BRAM0_ADDR_BASE, BRAM0_ADDR_RANGE)

# Access to memory map of the BRAM 1
BRAM1_ADDR_BASE = 0x42000000
BRAM1_ADDR_RANGE = 0x2000
bram1_obj = MMIO(BRAM1_ADDR_BASE, BRAM1_ADDR_RANGE)

In [2]:
# Input array
input_data = np.zeros((8,), dtype=np.uint32)

# Write input to array
for i in range(8):
    a = i + 1
    b = 8 - i
    y = i + 1
    input_data[i] = (y << 16) | (b << 8) | a

# Write array to BRAM 0
j = 0;
for i in range(0,8*4,4):
    bram0_obj.write(i, int(input_data[j]))
    j += 1

# Check BRAM 0
for i in range(0,8*4,4):
    print("0x%08X" % (bram0_obj.read(i)))

0x00010801
0x00020702
0x00030603
0x00040504
0x00050405
0x00060306
0x00070207
0x00080108


In [3]:
# Start the PE system
gpio0_obj.write(0, 1)
gpio0_obj.write(0, 0)
# Wait until ready is one
while (gpio0_obj.read(8) == 0):
    pass

In [4]:
# Output array
output_data = np.zeros((8,), dtype=np.uint32)

# Read BRAM 1 to array
j = 0;
for i in range(0,8*4,4):
    output_data[j] = bram1_obj.read(i)
    j += 1

# Check output array
for i in range(8):
    print("%d" % output_data[i])

9
16
21
24
25
24
21
16
